<a href="https://colab.research.google.com/github/fikhrimasri/Kaggle-House-Price/blob/master/Kaggle_House_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving DataTrain.csv to DataTrain (1).csv


In [0]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import warnings

# Load Data

In [0]:
house_data = pd.read_csv("DataTrain.csv")
house_data_test = pd.read_csv("DataTest.csv")

In [14]:
house_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Data Cleaning

In [0]:
train_parent=house_data
test_parent=house_data_test 
house_data = house_data.drop('Id', axis=1)
house_data_test = house_data_test.drop('Id', axis=1)

In [16]:
threshold=0.4 * len(house_data)
df=pd.DataFrame(len(house_data) - house_data.count(),columns=['count'])
df.index[df['count'] > threshold]

Index(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')

In [0]:
house_data = house_data.drop(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)
house_data_test = house_data_test.drop(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)

In [18]:
house_data['SalePrice'].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [0]:
house_data.select_dtypes(include=np.number).columns #will give all numeric columns ,we will remove the SalePrice column 
for col in ('MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'):
    
    house_data[col] = house_data[col].fillna(0)
    house_data_test[col] = house_data_test[col].fillna('0')

In [0]:
house_data.select_dtypes(exclude=np.number).columns
for col in ('MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition'):
    
    house_data[col] = house_data[col].fillna('None')
    house_data_test[col] = house_data_test[col].fillna('None')

In [21]:
house_data[house_data.isnull().any(axis=1)]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


In [22]:
house_data_test[house_data_test.isnull().any(axis=1)]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition


In [0]:
train=house_data
test=house_data_test

#Assigning a flag to training and testing dataset for segregation after OHE .
train['train']=1 
test['train']=0

#Combining training and testing dataset

combined=pd.concat([train,test])

In [0]:
#Applying One Hot Encoding to categorical data
ohe_data_frame=pd.get_dummies(combined, 
                           columns=['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition'],
      )

In [27]:
train_df=ohe_data_frame[ohe_data_frame['train']==1]
test_df=ohe_data_frame[ohe_data_frame['train']==0]
train_df.drop(['train'],axis=1,inplace=True)             
test_df.drop(['train','SalePrice'],axis=1,inplace=True)     

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [0]:
house_data=train_df
house_data_test=test_df

In [0]:
X_train = house_data.drop('SalePrice', axis=1)
Y_train = house_data['SalePrice']
X_test = house_data_test

In [31]:
num_estimators = [500,1000,3000]
learn_rates = [0.01, 0.02, 0.05, 0.1]
max_depths = [1, 2, 3, 4]
min_samples_leaf = [5,10,15]
min_samples_split = [2,5,10]

param_grid = {'n_estimators': num_estimators,
              'learning_rate': learn_rates,
              'max_depth': max_depths,
              'min_samples_leaf': min_samples_leaf,
              'min_samples_split': min_samples_split}

grid_search = GridSearchCV(GradientBoostingRegressor(loss='huber'),
                           param_grid, cv=3, return_train_score=True)
grid_search.fit(X_train, Y_train)

grid_search.best_params_  

{'learning_rate': 0.02,
 'max_depth': 2,
 'min_samples_leaf': 15,
 'min_samples_split': 2,
 'n_estimators': 3000}

In [32]:
params = {'n_estimators': 3000, 'max_depth': 1, 'min_samples_leaf':15, 'min_samples_split':10, 
          'learning_rate': 0.05, 'loss': 'huber','max_features':'sqrt'}
gbr_model = GradientBoostingRegressor(**params)
gbr_model.fit(X_train, Y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='huber', max_depth=1,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=3000,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [33]:
gbr_model.score(X_train, Y_train)

0.9039387495423273

In [34]:
y_grad_predict = gbr_model.predict(X_test)
print(y_grad_predict)

[124927.24876014 161185.48008556 193236.16936815 ... 166984.66211752
 126895.23685363 221870.99117472]


In [35]:
#Submission 
my_submission = pd.DataFrame({'Id': test_parent.Id, 'SalePrice': y_grad_predict})
print(my_submission)

my_submission.to_csv('submission1.csv',index = False)

        Id      SalePrice
0     1461  124927.248760
1     1462  161185.480086
2     1463  193236.169368
3     1464  202292.482980
4     1465  202135.062773
5     1466  175349.489559
6     1467  174881.740271
7     1468  168909.982535
8     1469  189946.317765
9     1470  121009.167986
10    1471  180348.837092
11    1472   99263.179560
12    1473   99618.904434
13    1474  147897.362071
14    1475  114074.878646
15    1476  359007.371005
16    1477  260700.411126
17    1478  305594.549831
18    1479  295816.273417
19    1480  461668.054535
20    1481  300552.924959
21    1482  201312.812142
22    1483  169909.073085
23    1484  164957.322454
24    1485  190640.386564
25    1486  193058.850636
26    1487  313243.470679
27    1488  229422.879425
28    1489  210251.324516
29    1490  250760.601524
...    ...            ...
1429  2890   80674.695358
1430  2891  126234.790413
1431  2892   41924.048872
1432  2893   80542.644922
1433  2894   69592.285731
1434  2895  312915.709735
1435  2896  